In [11]:
from catboost.datasets import titanic
import pandas as pd

DATA = '../lab4/data/'

TEST_PATH = DATA + 'titanic.test.parquet.gzip'
TRAIN_PATH = DATA + 'titanic.train.parquet.gzip'

In [8]:
# Загрузка данных

titanic_train, titanic_test = titanic()

titanic_train.to_parquet(TRAIN_PATH)
titanic_test.to_parquet(TEST_PATH)

In [9]:
# Удаление лишних колонок

titanic_train = pd.read_parquet(TRAIN_PATH)
titanic_test = pd.read_parquet(TEST_PATH)

cols_to_drop = [col for col in titanic_test.columns if col not in ["Pclass", "Sex", "Age"]]

titanic_train.drop(cols_to_drop, axis=1).to_parquet(TRAIN_PATH)
titanic_test.drop(cols_to_drop, axis=1).to_parquet(TEST_PATH)

In [12]:
TEST = 'TEST'
TRAIN = 'TRAIN'

In [20]:
# Заполнение пропусок средним значением в Age

data = pd.concat([
    pd.read_parquet(TRAIN_PATH).assign(COL_TYPE=TEST), 
    pd.read_parquet(TEST_PATH).assign(COL_TYPE=TRAIN)
])

age_mean = data['Age'].mean()

data['Age'] = data['Age'].fillna(age_mean)

titanic_test, titanic_train = data[data["COL_TYPE"].eq(TEST)].drop('COL_TYPE', axis=1), \
  data[data["COL_TYPE"].eq(TRAIN)].drop('COL_TYPE', axis=1)

titanic_test.to_parquet(TEST_PATH)
titanic_train.to_parquet(TRAIN_PATH)

In [23]:
# OHE для колонки Sex

data = pd.concat([
    pd.read_parquet(TRAIN_PATH).assign(COL_TYPE=TEST), 
    pd.read_parquet(TEST_PATH).assign(COL_TYPE=TRAIN)
])

data = pd.get_dummies(data, columns=['Sex'])

titanic_test, titanic_train = data[data["COL_TYPE"].eq(TEST)].drop('COL_TYPE', axis=1), \
  data[data["COL_TYPE"].eq(TRAIN)].drop('COL_TYPE', axis=1)

titanic_test.to_parquet(TEST_PATH)
titanic_train.to_parquet(TRAIN_PATH)